In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from typing import Dict, List
import pyprind
from math import cos, pi, floor
from datetime import datetime

In [2]:
def parse_challenge(page):
    """
    Parse a challenge given by mmi and mavat's web servers, forcing us to solve
    some math stuff and send the result as a header to actually get the page.
    This logic is pretty much copied from https://github.com/R3dy/jigsaw-rails/blob/master/lib/breakbot.rb
    """
    top = page.split('<script>')[1].split('\n')
    challenge = top[1].split(';')[0].split('=')[1]
    challenge_id = top[2].split(';')[0].split('=')[1]
    return {'challenge': challenge, 'challenge_id': challenge_id, 'challenge_result': get_challenge_answer(challenge)}

def telegram_bot_sendtext(bot_message: str) -> None:
    """
    Send telegram msg for my bot
    """
    bot_token = '1172952527:AAGoM74Rx25DPBpmQhEwacs_AQ9GWI8Oybk'
    chat_id = "839266998"
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + chat_id + '&parse_mode=Markdown&text=' + bot_message
    requests.get(send_text)

def get_challenge_answer(challenge):
    """
    Solve the math part of the challenge and get the result
    """
    arr = list(challenge)
    last_digit = int(arr[-1])
    arr.sort()
    min_digit = int(arr[0])
    subvar1 = (2 * int(arr[2])) + int(arr[1])
    subvar2 = str(2 * int(arr[2])) + arr[1]
    power = ((int(arr[0]) * 1) + 2) ** int(arr[1])
    x = (int(challenge) * 3 + subvar1)
    y = cos(pi * subvar1)
    answer = x * y
    answer -= power
    answer += (min_digit - last_digit)
    answer = str(int(floor(answer))) + subvar2
    return answer

def get_racing_days(df: pd.DataFrame) -> List:
    """
    filter wednesday, saturday and sundays
    filter non-august
    """
    datelist = pd.date_range("2020/09/01", datetime.now()).tolist()
    datelist = [x.strftime("%Y/%m/%d") for x in datelist if x.dayofweek in (2,5,6) and x.month != 8]

    # remove existing dates
    datelist = [x for x in datelist if x not in list(df["date"])]
    
    return datelist

def make_hkjc_request(url: str) -> str:
    headers = {'User-agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1'}
    with requests.Session() as s:
        r = s.get(url, headers = headers)
        if 'X-AA-Challenge' in r.text:
            challenge = parse_challenge(r.text)
            cookies = s.get(url, headers={
                'X-AA-Challenge': challenge['challenge'],
                'X-AA-Challenge-ID': challenge['challenge_id'],
                'X-AA-Challenge-Result': challenge['challenge_result']
            }).cookies  
        else:
            cookies = r.cookies

        r = s.post(url, cookies = cookies)

    return r.text

def evaluate_course(title):
    clean_title = title.replace(" ","").lower()
    if "shatin" in clean_title:
        return "ST"
    
    elif "happyvalley" in clean_title:
        return "HV"
    
    else:
        return None


def scrape_race_day(date) -> Dict:
    html_list = []
    # first race
    url = f"https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate={date}"
    first_race = make_hkjc_request(url)
    soup = BeautifulSoup(first_race, "lxml")
    html_list.append(soup)

    # extract info
    num_of_races = len(soup.find("table", attrs = {"class": re.compile("f_fs12.+racecard$")}).find_all("a"))
    course = evaluate_course(str(soup.find("span", attrs = {"class": "f_fl f_fs13"})))
    overhead_text = soup.find("tr", attrs = {"class": re.compile("bg_blue.+font_wb$")}).get_text()
    first_race_no = re.search("\((\d+)\)", overhead_text).group(1)

    # extract info for all other races and append to html_list
    for i in range(2, num_of_races + 1):
        url = f"https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate={date}&Racecourse={course}&RaceNo={str(i)}"
        race_info = make_hkjc_request(url)
        soup = BeautifulSoup(race_info, "lxml")
        html_list.append(soup)

    # get horse_id_list
    horse_id_list = []
    for soup in html_list:
        try:
            single_race_id_list = list(pd.read_html(str(soup.find("div", attrs = {"class": "performance"})))[0]["Horse"])
            single_race_id_list = [re.search("\(([\w\d]+)\)", x).group(1) for x in single_race_id_list]
            horse_id_list.append(single_race_id_list)
        except ValueError:
            pass

    result_dict = {
        "date": date,
        "first_race_no": first_race_no,
        "horse_id_list": horse_id_list    
    }

    return result_dict



In [3]:
if __name__ == "__main__":
    """
    Just run this and it should automatically update all horse_id and racing days to the latest for 2020 season
    """
    
    # original horse_id_data
    df = pd.read_pickle(r"2020_season\1_horse_id_data.pkl")

    # remove duplicated racing days
    date_list = get_racing_days(df)

    bar = pyprind.ProgBar(len(date_list))
    for race_date in date_list:
        try:
            df_dict = scrape_race_day(race_date)
            df = df.append(df_dict, ignore_index = True)
            bar.update()
        except AttributeError:
            bar.update()
            pass

    df.to_pickle(r"2020_season\1_horse_id_data.pkl")

0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10
